In [131]:
    #importing packages
    import sklearn
    import pandas as pd
    import quandl , math , datetime
    import numpy as np
    from sklearn import preprocessing, cross_validation, svm
    from sklearn.linear_model import LinearRegression
    import matplotlib.pyplot as plt
    from matplotlib import style

    style.use('ggplot')

In [71]:
#loading the data frame df 
df = quandl.get('WIKI/GOOGL')

In [72]:
#Checking the values of the df
print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [73]:
# Creating new df with only a few wanted cloumns
df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]

In [74]:
#Creating new columns 
df['HL_PCT']=(df['Adj. High']-df['Adj. Close'])/df['Adj. Close']*100.0

In [75]:
# Percentage Change Column is created
df['PCT_change']=(df['Adj. Close']-df['Adj. Open'])/df['Adj. Open']*100.0

In [76]:
#Dataframe with only required columns
df = df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]

In [77]:
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [78]:
import math

In [79]:
forecast_col = 'Adj. Close'

In [80]:
forecast_out= int(math.ceil(0.01*len(df)))

In [81]:
df['label']=df[forecast_col].shift(-forecast_out)

In [82]:
df.dropna(inplace=True)

In [83]:
print(df.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume      label
Date                                                                
2004-08-19   50.322842  3.712563    0.324968   44659000.0  69.078238
2004-08-20   54.322689  0.710922    7.227007   22834300.0  67.839414
2004-08-23   54.869377  3.729433   -1.227880   18256100.0  68.912727
2004-08-24   52.597363  6.417469   -5.726357   15247300.0  70.668146
2004-08-25   53.164113  1.886792    1.183658    9188600.0  71.219849


In [84]:
print(df.tail())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume    label
Date                                                              
2018-01-30     1177.37  0.896914   -0.029718    1792602.0  1094.00
2018-01-31     1182.22  0.346805   -0.134312    1643877.0  1053.15
2018-02-01     1181.59  0.495942    0.476195    2774967.0  1026.55
2018-02-02     1119.20  1.081129   -0.729098    5798880.0  1054.09
2018-02-05     1068.76  4.325574   -2.893850    3742469.0  1006.94


In [119]:
X= np.array(df.drop(['label'],1))
y=np.array(df['label'])

X= preprocessing.scale(X)

y=np.array(df['label'])

print(len(X),len(y))

# to implemnet LinearRegressio
X_test,X_train,y_test,y_train=cross_validation.train_test_split(X,y,test_size=0.2)
clf= LinearRegression(n_jobs=-1)
clf.fit(X_train,y_train)
accuracy= clf.score(X_test,y_test)

print(accuracy)

In [117]:
print(forecast_out)

35


In [113]:
# to implement SVM algorithum
X_test,X_train,y_test,y_train=cross_validation.train_test_split(X,y,test_size=0.2)
clf= svm.SVR(kernel='poly')
clf.fit(X_train,y_train)
accuracy= clf.score(X_test,y_test)

In [114]:
print(accuracy)

0.39127144541715325


In [128]:
X= np.array(df.drop(['label'],1))
X= preprocessing.scale(X)
X_lately = X[-forecast_out:]
X=X[:-forecast_out]

df.dropna(inplace=True)
y=np.array(df['label'])
y=np.array(df['label'])

X_test,X_train,y_test,y_train=cross_validation.train_test_split(X,y,test_size=0.2)
clf= LinearRegression(n_jobs=-1)
clf.fit(X_train,y_train)
accuracy= clf.score(X_test,y_test)

forecast_set= clf.predict(X_lately)

print(forecast_set,accuracy,forecast_out)

df['Forecast']=np.nan

last_date= df.iloc[-1].name
last_unix= last_date.timestamp()
one_day = 86400
next_unix=last_unix+one_day


for i in forecast_set :
    next_date= datetime.datetime.fromtimestamp(next_unix)
    next_unix+=one_day
    
    df.loc[next_date]=[np.nan for _ in range(len(df.columns)-1)+[i]]    

ValueError: Found input variables with inconsistent numbers of samples: [3354, 3389]

In [130]:
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

KeyError: 'Forecast'